In [1]:
import os
import pickle
import requests
import numpy as np
import random

In [2]:
def make_addition_examples(pad=True):
    print('making examples of a+b=c')
    input_file_path = os.path.join(os.path.dirname(__file__), 'add_examples.txt')
    # if not os.path.exists(input_file_path):
    with open(input_file_path, 'w+') as f:
        for i in range(10000):
            a, b = random.randint(0,999), random.randint(0,999)
            c = a + b
            if pad:
                f.write(f"{a:03}+{b:03}={c:04}\n")
            else:
                f.write(f"{a}+{b}={c}\n")


In [27]:
def pad_zero_digit(a: int) -> str:
    a = f'{a:03}'
    return "0"+"0".join(a)


def remove_zero_pad(a: str):
    return a[1::2]


def make_addition_examples_zero_padded(pad=True):
    # we have all (=100) 1 digit addition examples + some (=1000) 2 digit addition examples
    # so we have 1 digit : 100 / 2 digit: 1000-100 / 3 digit: 9000
    print('making examples of a+b=c')
    input_file_path = 'add_examples.txt'
    # if not os.path.exists(input_file_path):
    count = 0
    count_2 = 0
    with open(input_file_path, 'w+') as f:
        for a in range(1000):
            for b in range(1000):
                if a < 10 and b < 10: p = 1
                elif a < 100 and b < 100: p = 900/9900
                else: p = 9000/990,000
                
                pp = random.uniform(0,1)
                if pp > p:
                    continue

                if a < 100 and b < 100:
                    count_2 += 1
                
                c = a + b
                aa, bb, cc = pad_zero_digit(a), pad_zero_digit(b), pad_zero_digit(c)
                f.write(f"{aa}+{bb}={cc}\n")
            
                count += 1
                if count == 10000:
                    print(f'{count_2} - 2 digit numbers')
                    print(f'{count} number of samples saved to {input_file_path}')
                    return
                    
    print(f'{count_2} - 2 digit numbers')
    print(f'{count} number of samples saved to {input_file_path}')


In [35]:
def make_addition_examples_v2(pad=True):
    print('making examples of a+b=c')
    input_file_path = 'add_examples.txt'
    # if not os.path.exists(input_file_path):
    count = 0
    num_two_digit = 0
    with open(input_file_path, 'w+') as f:
        for a in range(1000):
            for b in range(1000):
                if a < 10 and b < 10: p = 100/100
                elif a < 100 and b < 100: p = 900/(100*100-10*10)
                else: p = 9000/(1000*1000-100*100)
                
                pp = random.uniform(0,1)
                if pp > p:
                    continue
                
                if a < 100 and b < 100:
                    num_two_digit +=1 
                    
                c = a + b
                
                if pad:
                    f.write(f"{a:03}+{b:03}={c:04}\n")
                else:
                    f.write(f"{a}+{b}={c}\n")

            
                count += 1
                if count == 10000:
                    print(f'{num_two_digit} - 2 digit numbers')
                    print(f'{count} number of samples saved to {input_file_path}')
                    return
                    
    print(f'{num_two_digit} - 2 digit numbers')
    print(f'{count} number of samples saved to {input_file_path}')



In [99]:
# download the tiny shakespeare dataset
input_file_path =  'add_examples.txt'
# if not os.path.exists(input_file_path):
#     make_addition_examples_v2(pad=True)
make_addition_examples_v2(pad=True)

with open(input_file_path, 'r') as f:
    data = f.read()
print(f"length of dataset in characters: {len(data):,}")

# get all the unique characters that occur in this text
chars = sorted(list(set(data)))
vocab_size = len(chars)
print("all the unique characters:", ''.join(chars))
print(f"vocab size: {vocab_size:,}")

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
def encode(s):
    return [stoi[c] for c in s] # encoder: take a string, output a list of integers
def decode(l):
    ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# create the train and test splits
n = len(data) # 130,023
train_data = data[:int(n*0.9)]
val_data = data[int(n*0.9):]

# encode both to integers
train_ids = encode(train_data)
val_ids = encode(val_data)
print(f"train has {len(train_ids):,} tokens")
print(f"val has {len(val_ids):,} tokens")

# export to bin files
train_ids = np.array(train_ids, dtype=np.uint16)
val_ids = np.array(val_ids, dtype=np.uint16)
train_ids.tofile(f'train.bin')
val_ids.tofile(f'val.bin')

# save the meta information as well, to help us encode/decode later
meta = {
    'vocab_size': vocab_size,
    'itos': itos,
    'stoi': stoi,
}
with open(f'meta.pkl', 'wb') as f:
    pickle.dump(meta, f)


making examples of a+b=c
998 - 2 digit numbers
10000 number of samples saved to add_examples.txt
length of dataset in characters: 130,000
all the unique characters: 
+0123456789=
vocab size: 13
train has 117,000 tokens
val has 13,000 tokens


In [3]:
with open("add_examples.txt", "r") as f:
    lines = []
    for line in f:
        a = line.split("+")[0]
        b = line.split("+")[1].split("=")[0]
        c = line.split("=")[1]
        a, b, c = int(a), int(b), int(c)
        lines.append(f"{a}+{b}={c}\n")

# shuffle and save
if not os.path.exists("add_examples_no_pad.txt"):
    with open("add_examples_no_pad.txt", "w") as f2:
        random.shuffle(lines)
        for line in lines:
            f2.write(line)

In [6]:
input_file_path = 'add_examples_no_pad.txt'
with open(input_file_path, 'r') as f:
    data = f.read()
print(f"length of dataset in characters: {len(data):,}")

# get all the unique characters that occur in this text
chars = sorted(list(set(data)))
vocab_size = len(chars)
print("all the unique characters:", ''.join(chars))
print(f"vocab size: {vocab_size:,}")

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
def encode(s):
    return [stoi[c] for c in s] # encoder: take a string, output a list of integers
def decode(l):
    ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# create the train and test splits
n = len(data)
train_data = data
# train_data = data[:int(n*0.9)]
# val_data = data[int(n*0.9):]

# encode both to integers
train_ids = encode(train_data)
# val_ids = encode(val_data)
print(f"train has {len(train_ids):,} tokens")
# print(f"val has {len(val_ids):,} tokens")

# export to bin files
train_ids = np.array(train_ids, dtype=np.uint16)
# val_ids = np.array(val_ids, dtype=np.uint16)
train_ids.tofile('train_no_pad.bin')
# val_ids.tofile(os.path.join(os.path.dirname(__file__), 'val.bin'))

# save the meta information as well, to help us encode/decode later
meta = {
    'vocab_size': vocab_size,
    'itos': itos,
    'stoi': stoi,
}
with open('meta.pkl', 'wb') as f:
    pickle.dump(meta, f)


length of dataset in characters: 119,843
all the unique characters: 
+0123456789=
vocab size: 13
train has 119,843 tokens
